https://docs.scipy.org/doc/scipy/reference/cluster.vq.html


## Word Distribution Cluster Analysis

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os

root_folder = os.path.abspath(os.path.join(globals()['_dh'][0], "../../.."))
sys.path = [ root_folder ] + sys.path

import ipywidgets
import bokeh

from bokeh.plotting import figure, output_notebook, show
from beakerx import *
from beakerx.object import beakerx

import holoviews as hv

from IPython.display import display

import westac.common.goodness_of_fit as gof
import westac.corpus.vectorized_corpus as vectorized_corpus

output_notebook()

hv.extension('bokeh')


## Load previously vectorized corpus

The corpus was created using the following settings:
 - Tokens were converted to lower case.
 - Only tokens that contains at least one alphanumeric character (isalnum).
 - Accents are ot removed (deacc)
 - Min token length 2 (min_len)
 - Max length not set (max_len)
 - Numerals are removed (numerals, -N)
 - Symbols are removed (symbols, -S)

Use the `corpus_vectorizer` module to create a new corpus with different settings.

The loaded corpus is processed in the following ways:

 - Exclude tokens having a total word count less than 10000
 - Include at most 50000 most frequent words words.
 - Group documents by year (y_corpus).
 - Normalize token count based on each year's total token count. 
 - Normalize token distrubution over years to 1.0 (n_corpus).


In [ ]:

y_corpus = vectorized_corpus\
    .load_corpus('SOU_test_L0_+N_+S', os.path.join(root_folder, 'output'), n_count=5000, n_top=50000, axis=1, keep_magnitude=False)

n_corpus = y_corpus.normalize(axis=0)


## Deviation metrics

These metrics are used to identify tokens that have a distribution that deviates the most to a uniform distribution.
The following metrics are computed for each token:

| Metric |  | |
| :------ | :------- | :------ |
| L2-norm | Measures distance to unform distribution. Lower bound 1/sqrt(d) is uniformity and upper bound is a 1-hot vector. |
| Linear regression | Curve fitting to f(x) = k * x + m, where k is the slope, and m is the intercept to y axis when x is 0. A uniform curve has slope equals to 0. |
| χ2 test | [Khan](https://www.khanacademy.org/math/statistics-probability/inference-categorical-data-chi-square-tests/chi-square-goodness-of-fit-tests/v/pearson-s-chi-square-test-goodness-of-fit) [Blog](https://towardsdatascience.com/inferential-statistic-understanding-hypothesis-testing-using-chi-square-test-eacf9fcac533) [Wikipedia](https://en.wikipedia.org/wiki/Chi-squared_test) |
| Statistics | The min, max, mean and variance of the distribution |
| Earth mover distance | [PDF](http://infolab.stanford.edu/pub/cstr/reports/cs/tr/99/1620/CS-TR-99-1620.ch4.pdf) [Wikipedia](https://en.wikipedia.org/wiki/Earth_mover%27s_distance)  |
| Kullback-Leibler divergence | S = sum(pk * log(pk / qk), where pk is the token distribution and qk is the uniform distribution |
| Entropy | Basically the same as KLD |
| Skew | A measure of the "skewness" of the token distribution. See [scipy.stats.skew](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skew.html#scipy.stats.skew) |

References:

 - [StackExchange](stats.stackexchange.com/questions/25827/how-does-one-measure-the-non-uniformity-of-a-distribution)
    "It just so happens, though, that the L2 norm has a simple algebraic connection to the χ2 statistic used in goodness of fit tests:
    that's the reason it might be suitable to measure non-uniformity"


In [ ]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

def compute_uniformity_metrics(x_corpus, n_count=2000):

    df_gof = gof.compute_goddness_of_fits_to_uniform(x_corpus)
    df_most_deviating = gof.compile_most_deviating_words(df_gof, n_count=n_count)

    return df_gof, df_most_deviating

def display_uniformity_metrics(x_corpus, df_gof, df_most_deviating):
    
    output_row = [ ipywidgets.Output(), ipywidgets.Output() ]
    display(ipywidgets.HBox(output_row))
    
    with output_row[0]:
        
        columns = ['token', 'word_count', 'l2_norm', 'slope', 'chi2_stats', 'earth_mover', 'kld', 'skew']
        display(df_gof.nlargest(10000, columns=['word_count'])[columns])
        
    with output_row[1]:
        pass
        #columns = ['L2 token', 'L2', 'K token', 'K', 'X2 token', 'X2', 'EMD token', 'EMD', 'KLD token', 'KLD', 'Ent. token', 'Entropy']
        #df_most_deviating.columns = columns
        #display(df_most_deviating.head(100))
    
    gof.plot_metrics(df_gof)
    gof.plot_slopes(x_corpus, df_most_deviating, 'l2_norm')

    # df_gof.to_csv('df_gof.txt', sep='\t')
    # df_most_deviating.to_csv('df_most_deviating.txt', sep='\t')


df_gof, df_most_deviating = compute_uniformity_metrics(n_corpus, n_count=10000)

display_uniformity_metrics(n_corpus, df_gof, df_most_deviating)



## Word distributions

In [ ]:
import westac.notebooks_gui.distributions_plot_gui as pdg

tokens = df_most_deviating['l2_norm_token']
pdg.display_gui(n_corpus, tokens)


### Cluster Analysis (Hierarchical Clustering & K-means)


In [ ]:
import westac.notebooks_gui.cluster_analysis_gui as cluster_analysis_gui

container = cluster_analysis_gui.display_gui(n_corpus, df_gof)



### Some references

|  |  |
| :------- | :------- |
| Overview: | https://docs.scipy.org/doc/scipy-0.19.0/reference/cluster.hierarchy.html |
| Linkage: |https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage |
| Reference | Daniel Mullner, “Modern hierarchical, agglomerative clustering algorithms”, [arXiv:1109.2378v1](https://arxiv.org/abs/1109.2378v1). |
|  | [Link](https://stackabuse.com/hierarchical-clustering-with-python-and-scikit-learn/) [Link](https://github.com/giusdp/Clustering-Techniques/tree/0c78d1a893995c4603ed07216d645801ab4fdb4d)
